REFRENCE: [link text](https://www.youtube.com/watch?v=4FmVIpcwl4k&list=RDCMUCwBs8TLOogwyGd0GxHCp-Dw&index=4)

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler , StandardScaler
import tensorflow as tf


In [38]:
data=pd.read_csv("Load.csv" )
data['Date'] = pd.to_datetime(data['Date'])
dataset=data.set_index(data['Date'], drop=False, append=False, inplace=False, verify_integrity=False).drop('Date', 1)
dataset.head(2)

<ipython-input-38-73b4da23f7b7>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataset=data.set_index(data['Date'], drop=False, append=False, inplace=False, verify_integrity=False).drop('Date', 1)


,Load
Date,
2016-01-01,1158.8
2016-01-01,1576.1


In [40]:
#data_input = data[["" , ""]]   #ALL the columns (feature & target)
data_input = data[["Load"]]

In [41]:
#data_input.shape

In [42]:
#data_input

In [43]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_input)

In [44]:
features = data_scaled
target = data_scaled[:,0]  # 0 index

Just for see how TimeseriesGenerator works:

In [45]:
#TimeseriesGenerator(features, target,
       #             length=2, # obs is hourly --> taking 2hours as input : """""2ROWS"""""
        #           sampling_rate=1,
  #                  batch_size=1
        #            )[0]

In [46]:
x_train, x_test, y_train, y_test = train_test_split(features, target,test_size=0.20, random_state=123, 
                                                    shuffle=False # MUST(!) for TS data (itd deafult=true)
                                                    )

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(308118, 1) (77030, 1) (308118,) (77030,)


Now the actual TimeseriesGenerator:

In [47]:
win_length = 12 
batch_size= 32  
num_features= 1
train_generator= TimeseriesGenerator(x_train, y_train, length= win_length,sampling_rate=1, batch_size=batch_size )
test_generator= TimeseriesGenerator(x_test, y_test, length= win_length,sampling_rate=1, batch_size=batch_size )

In [ ]:
#train_generator[0]

In [49]:
#train_generator


In [50]:
n_timesteps_in=win_length
n_features=num_features
numberOfLSTMunits=16

In [51]:
from tensorflow import keras

from keras import Input, models
from keras.models import Sequential, Model

from keras.layers import LSTM, Dense, TimeDistributed, RepeatVector
from keras.callbacks import EarlyStopping

from keras.layers import Lambda
from keras import backend as K

In [52]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(16, input_shape= (win_length, num_features)
#, return_sequences=True 
))
model.add(tf.keras.layers.Dense(1))

In [53]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 16)                1152      
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,169
Trainable params: 1,169
Non-trainable params: 0
_________________________________________________________________


In [54]:
model.compile(loss=tf.losses.MeanSquaredError(),
              optimizer=tf.optimizers.Adam(),
              metrics=[tf.metrics.MeanAbsoluteError()])

In [55]:
history= model.fit_generator(train_generator, 
             validation_data=test_generator ,
             shuffle=False,                          
          #batch_size=batch_size,
          epochs=2,
          )

Epoch 1/2


<ipython-input-55-283f57de9620>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history= model.fit_generator(train_generator,


9629/9629 [==============================] - 55s 6ms/step - loss: 0.0055 - mean_absolute_error: 0.0467 - val_loss: 0.0030 - val_mean_absolute_error: 0.0308
Epoch 2/2
9629/9629 [==============================] - 53s 6ms/step - loss: 0.0021 - mean_absolute_error: 0.0233 - val_loss: 0.0024 - val_mean_absolute_error: 0.0250


In [56]:
model.evaluate_generator(test_generator, verbose=0)

<ipython-input-56-a68ec1088d46>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model.evaluate_generator(test_generator, verbose=0)


[0.002360998885706067, 0.025013383477926254]

In [57]:
predictions=model.predict_generator(test_generator)

<ipython-input-57-6ca49477046f>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions=model.predict_generator(test_generator)


In [58]:
#predictions.shape[0]

77018

In [60]:
#x_test[:,1:][win_length:]

array([], shape=(77018, 0), dtype=float64)

In [61]:
def_pred=pd.concat([pd.DataFrame(predictions), pd.DataFrame(x_test[:,1:][win_length:])], axis=1)

In [62]:
rev_trans=scaler.inverse_transform(def_pred)

In [63]:
df_final=data_input[predictions.shape[0]*-1:]

In [64]:
#df_final.count()

Load    77018
dtype: int64

In [65]:
df_final["Load_pred"]=rev_trans[:,0]

<ipython-input-65-89ef7dce4363>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["Load_pred"]=rev_trans[:,0]


In [66]:
df_final

,Load,Load_pred
308130,1769.2,1760.203003
308131,680.7,437.147156
308132,1129.8,886.039062
308133,1033.2,864.474182
308134,2208.3,1830.579712
...,...,...
385143,835.6,699.781311
385144,309.9,324.303833
385145,5063.4,5484.648926
385146,575.5,652.604797
